In [1]:
import keras
import numpy
import pandas
from pathlib import Path

In [ ]:
for chip_size in [8, 16, 32, 64]:
    for arch in ["image", "multi", "slided"]:
        app = f"_{arch}" if arch != "image" else ""
        specs = {
            'chips': f"../../chips_gb/v2_{chip_size}{app}.npy",
        }
        chips = numpy.load(specs["chips"])
        p = Path(specs['chips'])
        model = keras.models.load_model(f"../../ai/v2_gb_{p.stem[3:]}/model/efficientnet_pooling_256_12_best/")
        prediction = model.predict(chips)
        numpy.save(p.parent.joinpath(p.stem + '_prediction.npy'), prediction)
        print(f"{chip_size}, {arch} done")

2022-08-11 08:52:08.962512: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6619 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:21:00.0, compute capability: 7.5
2022-08-11 08:52:28.256477: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


8, image done
8, multi done
8, slided done
16, image done
16, multi done
16, slided done
32, image done
32, multi done


2022-08-11 11:54:26.309065: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2769524736 exceeds 10% of free system memory.
2022-08-11 11:54:27.750516: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2769524736 exceeds 10% of free system memory.


32, slided done
64, image done
64, multi done


2022-08-11 12:51:51.835919: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4585033728 exceeds 10% of free system memory.
2022-08-11 12:51:54.231858: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4585033728 exceeds 10% of free system memory.


In [3]:
model = keras.models.load_model(f"../../ai/v2_gb_{p.stem[3:]}/model/efficientnet_pooling_256_12_best/")

In [4]:
model.output_names

['dense_7']

In [5]:
model.inputs

[<KerasTensor: shape=(None, 64, 64, 3) dtype=float32 (created by layer 'input_8')>]

In [8]:
    from model_builder import model_builder, relabel, class_merger, balancer
    import tensorflow as tf

    
    chip_size = 32
    
    specs = {
        'chips': f"../../chips_gb/v2_{chip_size}.npy",
        'labels': f"../../chips_gb/v2_{chip_size}_labels.parquet",
        'folder': f"../../ai/v2_gb_{chip_size}_slided/",
    }

    chips = numpy.load(specs['chips'])
    labels = pandas.read_parquet(specs['labels'], columns=['signature_type', 'split'])

    sorter = numpy.arange(0, chips.shape[0])
    numpy.random.shuffle(sorter)
    chips = chips[sorter]
    labels = labels.iloc[sorter].reset_index(drop=True)

    group_mapping = [
        ['9_0', '9_1', '9_2', '9_4', '9_5'],
        ['2_0'], 
        ['2_1'], 
        ['2_2'],
        ['1_0'], 
        ['3_0'], 
        ['5_0'], 
        ['6_0'], 
        ['8_0'],
        ['0_0'],
        ['4_0'],
        ['7_0']
    ]

    group_naming = [
        "Urbanity", 
        "Dense residential neighbourhoods",
        "Connected residential neighbourhoods",
        "Dense urban neighbourhoods",
        "Accessible suburbia",
        "Open sprawl",
        "Warehouse_Park land",
        "Gridded residential quarters",
        "Disconnected suburbia",
        "Countryside agriculture", 
        "Wild countryside", 
        "Urban buffer"
    ]

    mask = labels.split.str.startswith('nn')
    labels_nn = labels[mask]
    chips_nn = chips[mask]

    labels_merged = class_merger(labels_nn.signature_type, group_mapping)

    train_mask = labels_nn['split'] == "nn_train"

    train_dataset = tf.data.Dataset.from_tensor_slices((chips_nn[train_mask], labels_merged[train_mask]))

In [9]:
labels_merged[train_mask]

array([ 9, 10,  6, ..., 11, 11, 10])